In [1]:
import json

import pandas as pd
import numpy as np

import utils
from tfidf_model import TFIDFModel
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from tfidf_model import TFIDFModel
import json
from scipy import sparse

In [26]:
custom_names = [
    utils.COL_BOOK_SIMILARITY,
    utils.COL_CONTAINS_EMOTICON,
    utils.COL_CONTAINS_LINK,
    utils.COL_WORD_COUNT,
    utils.COL_CHAR_COUNT,
    utils.COL_UPPERCASE_COUNT,
    utils.COL_QUESTION_COUNT,
]

In [3]:
data_df, target_df = utils.load_discussions_data()

In [8]:
for reg in [0.15]:
    tfidf = TFIDFModel(max_iter=2500, regularization=reg, solver="sag", custom_feature_names=custom_names)
    evals = utils.evaluate_models([tfidf], data_df, target_df)
    print(json.dumps(evals, indent="  "))

{
  "TFIDFModel(custom_feature_names=['Book Similarity', 'Contains Emoticon',\n                                 'Contains Link', 'Word Count', 'Char Count',\n                                 'Uppercase Count'],\n           max_iter=2500, regularization=0.15, solver='sag')": {
    "fit_time": 4.660197019577026,
    "score_time": 0.11431044340133667,
    "test_accuracy": 0.6955879034078086,
    "test_f1_macro": 0.4613098165942646,
    "test_neg_log_loss": -1.082594111621562
  }
}


In [27]:
tfidf = TFIDFModel(max_iter=2500, regularization=reg, solver="sag", custom_feature_names=custom_names)
tfidf.fit(data_df, target_df)

TFIDFModel(custom_feature_names=['Book Similarity', 'Contains Emoticon',
                                 'Contains Link', 'Word Count', 'Char Count',
                                 'Uppercase Count', 'Question Words Count'],
           max_iter=2500, regularization=0.15, solver='sag')

In [44]:
max_coefs = tfidf._log_reg.coef_.max(axis=0)[:len(custom_names)]
best_class_idxs = tfidf._log_reg.coef_.argmax(axis=0)[:len(custom_names)]
best_classes = tfidf._log_reg.classes_[best_class_idxs]
custom_features_df = pd.DataFrame(list(zip(custom_names, max_coefs, best_classes)), columns=["Custom Feature", "Max Coefficient", "Predictive Class"], )

In [54]:
custom_features_df.sort_values(by="Max Coefficient", ascending=False, inplace=True)
custom_features_df

,Custom Feature,Max Coefficient,Predictive Class
4,Char Count,3.032205,Content Discussion
0,Book Similarity,2.560155,Content Discussion
3,Word Count,1.202205,Assignment Instructions
5,Uppercase Count,1.118973,Greeting
6,Question Words Count,1.096541,Content Question
2,Contains Link,0.687510,Outside Material
1,Contains Emoticon,0.485253,Emoticon/Non-verbal


In [55]:
custom_features_df.to_latex()

'\\begin{tabular}{llrl}\n\\toprule\n{} &        Custom Feature &  Max Coefficient &         Predictive Class \\\\\n\\midrule\n4 &            Char Count &         3.032205 &       Content Discussion \\\\\n0 &       Book Similarity &         2.560155 &       Content Discussion \\\\\n3 &            Word Count &         1.202205 &  Assignment Instructions \\\\\n5 &       Uppercase Count &         1.118973 &                 Greeting \\\\\n6 &  Question Words Count &         1.096541 &         Content Question \\\\\n2 &         Contains Link &         0.687510 &         Outside Material \\\\\n1 &     Contains Emoticon &         0.485253 &      Emoticon/Non-verbal \\\\\n\\bottomrule\n\\end{tabular}\n'